In [1]:
import sys
import os

custom_path = "../../../src"
if custom_path not in sys.path:
    sys.path.insert(0, custom_path)

In [2]:
import datetime
import pandas as pd
from socio4health import Extractor
from socio4health.harmonizer import Harmonizer
from socio4health.utils import harmonizer_utils, extractor_utils

In [ ]:
raw_dic = pd.read_excel("../../../../Socio4HealthData/Dictionaries/Brazil/Diccionario Crudo Censo2.xlsx")
dic=harmonizer_utils.standardize_dict(raw_dic)
colnames, colspecs =extractor_utils.parse_fwf_dict(dic)
dic

c:\Users\Juan\OneDrive - Universidad de los andes\Uniandes\2025\Harmonize\socio4health\docs\source\notebooks\../../../src\socio4health\utils\harmonizer_utils.py:99: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_process_group, include_groups=True)\


,variable_name,question,description,value,initial_position,final_position,size,dec,type,possible_answers
0,V0402,a responsabilidade pelo domicílio é de:,NaN,1.0; 2.0; 9.0,107.0,107.0,1.0,NaN,C,apenas um morador; mais de um morador; ignorado
1,V0209,"abastecimento de água, canalização:",NaN,1.0; 2.0; 3.0,90.0,90.0,1.0,NaN,C,"sim, em pelo menos um cômodo; sim, só na propr..."
2,V0208,"abastecimento de água, forma:",NaN,1.0; 2.0; 3.0; 4.0; 5.0; 6.0; 7.0; 8.0; 9.0; 10.0,88.0,89.0,2.0,NaN,C,rede geral de distribuição; poço ou nascente n...
3,V6210,adequação da moradia,NaN,1.0; 2.0; 3.0,144.0,144.0,1.0,NaN,C,adequada; semi-adequada; inadequada
4,V0301,alguma pessoa que morava com você(s) estava mo...,NaN,1.0; 2.0,104.0,104.0,1.0,NaN,C,sim; não
...,...,...,...,...,...,...,...,...,...,...
71,V0214,"televisão, existência:",NaN,1.0; 2.0,95.0,95.0,1.0,NaN,C,sim; não
72,V4002,tipo de espécie:,NaN,11.0; 12.0; 13.0; 14.0; 15.0; 51.0; 52.0; 53.0...,56.0,57.0,2.0,NaN,C\n,casa; casa de vila ou em condomínio; apartamen...
73,V0001,unidade da federação:,NaN,11.0; 12.0; 13.0; 14.0; 15.0; 16.0; 17.0; 21.0...,1.0,2.0,2.0,NaN,A,rondônia; acre; amazonas; roraima; pará; amapá...
74,V2011,valor do aluguel (em reais),NaN,NaN,59.0,64.0,6.0,NaN,N,NaN


In [ ]:
bra_online_extractor = Extractor(input_path="https://www.ibge.gov.br/estatisticas/sociais/saude/9662-censo-demografico-2010.html?=&t=microdados",
                                 down_ext=['.txt', '.csv', '.zip'],
                                 output_path="../IBGE2010_",
                                 key_words=["^[A-Z]+\.zip$"],
                                 depth=0, colnames=colnames, colspecs=colspecs)
bra_Censo_2010 = bra_online_extractor.extract()

<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
C:\Users\jmont\AppData\Local\Temp\ipykernel_21212\4208190101.py:4: SyntaxWarning: invalid escape sequence '\.'
  key_words=["^[A-Z]+\.zip$"],
2025-08-28 12:03:58,320 - INFO - ----------------------
2025-08-28 12:03:58,321 - INFO - Starting data extraction...
2025-08-28 12:03:58,322 - INFO - Extracting data in online mode...
2025-08-28 12:03:58,322 - INFO - Scraping URL: https://www.ibge.gov.br/estatisticas/sociais/saude/9662-censo-demografico-2010.html?=&t=microdados with depth 0
2025-08-28 12:04:53,858 - INFO - Spider completed successfully for URL: https://www.ibge.gov.br/estatisticas/sociais/saude/9662-censo-demografico-2010.html?=&t=microdados
2025-08-28 12:04:53,900 - INFO - Downloading files to: ../IBGE2010_
2025-08-28 13:19:45,295 - INFO - Processing (depth 0): RO.zip
2025-08-28 13:19:45,687 - WARNING - No matches in RO.zip. Contents:
2025-08-28 13:19:45,687 - WARNING - No files f

ValueError: Extraction failed: No data could be extracted from downloaded files

In [7]:
bra_extractor = Extractor(input_path="../IBGE2010_",down_ext=['.txt','.zip'],is_fwf=True,output_path="../IBGE2010_", colnames=colnames, colspecs=colspecs)
bra_Censo_2010 = bra_extractor.extract()

2025-08-31 11:10:47,944 - INFO - ----------------------
2025-08-31 11:10:47,944 - INFO - Starting data extraction...
2025-08-31 11:10:47,945 - INFO - Extracting data in local mode...
2025-08-31 11:10:47,948 - INFO - Processing (depth 0): AC.zip
2025-08-31 11:10:48,331 - INFO - Extracted: 1b681951_AC_Dom12.txt
2025-08-31 11:10:48,348 - INFO - Extracted: 1b681951_AC_FAMI12.TXT
2025-08-31 11:10:48,368 - INFO - Extracted: 1b681951_AC_Pes12.txt
2025-08-31 11:10:48,368 - INFO - Processing (depth 0): AL.zip
2025-08-31 11:10:50,650 - INFO - Extracted: 06d083ad_AL_DOM27.txt
2025-08-31 11:10:50,656 - INFO - Extracted: 06d083ad_AL_FAMI27.TXT
2025-08-31 11:10:50,707 - INFO - Extracted: 06d083ad_AL_PES27.txt
2025-08-31 11:10:50,709 - INFO - Processing (depth 0): AM.zip
2025-08-31 11:10:51,262 - INFO - Extracted: 3baa1695_AM_Dom13.txt
2025-08-31 11:10:51,268 - INFO - Extracted: 3baa1695_AM_FAMI13.TXT
2025-08-31 11:10:51,315 - INFO - Extracted: 3baa1695_AM_Pes13.txt
2025-08-31 11:10:51,317 - INFO - P

In [ ]:
har = Harmonizer()
har.similarity_threshold = 0.9
dfs = har.vertical_merge(bra_Censo_2010)

Merging groups: 100%|██████████| 1/1 [00:02<00:00,  2.28s/it]


DataFrame shape: (<dask_expr.expr.Scalar: expr=(Concat(frames=[Assign(frame=ArrowStringConversion(frame=FromMapProjectable(281b8c7)))[['V0402', 'V0209', 'V0208', 'V6210', 'V0301', 'V2012', 'V0222', 'V0205', 'V0300', 'V1002', 'V1003', 'V1004', 'V0002', 'V0204', 'V0203', 'V0701', 'V6204', 'V6203', 'V0201', 'V0211', 'V0207', 'V6600', 'V4001', 'V0212', 'V0216', 'V0210', 'M0201', 'M0202', 'M0203', 'M0204', 'M0205', 'M0206', 'M0207', 'M0208', 'M0209', 'M0210', 'M0211', 'M0212', 'M0213', 'M0214', 'M0215', 'M0216', 'M0217', 'M0218', 'M0219', 'M0220', 'M0221', 'M0222', 'M0301', 'M0401', 'M0402', 'M0701', 'M2011', 'V0202', 'V0220', 'V0219', 'V0221', 'V0215', 'V0010', 'V0401', 'V1001', 'V6531', 'V6532', 'V6530', 'V6529', 'V0213', 'V0206', 'V1006', 'V1005', 'V0217', 'V0218', 'V0214', 'V4002', 'V0001', 'V2011', 'V0011', 'filename']], Assign(frame=ArrowStringConversion(frame=FromMapProjectable(1423694)))[['V0402', 'V0209', 'V0208', 'V6210', 'V0301', 'V2012', 'V0222', 'V0205', 'V0300', 'V1002', 'V100

In [ ]:
dic = harmonizer_utils.translate_column(dic, "question", language="en")
dic = harmonizer_utils.translate_column(dic, "description", language="en")
dic = harmonizer_utils.translate_column(dic, "possible_answers", language="en")
dic = harmonizer_utils.classify_rows(dic, "question_en", "description_en", "possible_answers_en",
                                        new_column_name="category",
                                        MODEL_PATH="../../../../Socio4HealthData/input/bert_finetuned_classifier")
dic

,variable_name,question,description,value,initial_position,final_position,size,dec,type,possible_answers,question_en,description_en,possible_answers_en,category
0,V0402,a responsabilidade pelo domicílio é de:,NaN,1.0; 2.0; 9.0,107.0,107.0,1.0,NaN,C,apenas um morador; mais de um morador; ignorado,The responsibility for the home is:,NaN,just a resident; more than one resident; ignored,Housing
1,V0209,"abastecimento de água, canalização:",NaN,1.0; 2.0; 3.0,90.0,90.0,1.0,NaN,C,"sim, em pelo menos um cômodo; sim, só na propr...","water supply, channeling:",NaN,"Yes, in at least one room; Yes, only on the pr...",Housing
2,V0208,"abastecimento de água, forma:",NaN,1.0; 2.0; 3.0; 4.0; 5.0; 6.0; 7.0; 8.0; 9.0; 10.0,88.0,89.0,2.0,NaN,C,rede geral de distribuição; poço ou nascente n...,"water supply, form:",NaN,General Distribution Network; well or source o...,Business
3,V6210,adequação da moradia,NaN,1.0; 2.0; 3.0,144.0,144.0,1.0,NaN,C,adequada; semi-adequada; inadequada,Housing Adequacy,NaN,adequate; semi-adherence; inadequate,Housing
4,V0301,alguma pessoa que morava com você(s) estava mo...,NaN,1.0; 2.0,104.0,104.0,1.0,NaN,C,sim; não,Someone who lived with you (s) was living in a...,NaN,Yes; no,Business
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,V0214,"televisão, existência:",NaN,1.0; 2.0,95.0,95.0,1.0,NaN,C,sim; não,"television, existence:",NaN,Yes; no,Identification
72,V4002,tipo de espécie:,NaN,11.0; 12.0; 13.0; 14.0; 15.0; 51.0; 52.0; 53.0...,56.0,57.0,2.0,NaN,C\n,casa; casa de vila ou em condomínio; apartamen...,Type of species:,NaN,home; village house or condominium; apartment;...,Housing
73,V0001,unidade da federação:,NaN,11.0; 12.0; 13.0; 14.0; 15.0; 16.0; 17.0; 21.0...,1.0,2.0,2.0,NaN,A,rondônia; acre; amazonas; roraima; pará; amapá...,Federation unit:,NaN,Rondônia; acre; Amazonas; Roraima; to; Amapá; ...,Business
74,V2011,valor do aluguel (em reais),NaN,NaN,59.0,64.0,6.0,NaN,N,NaN,Rental value (in reais),NaN,NaN,Business


In [14]:
har.dict_df = dic
har.categories = ["Housing"]
har.key_col = 'V0002'
filtered_ddfs = har.data_selector(dfs)

2025-08-31 12:27:41,711 - WARNING - No rows found matching key values in DataFrame


In [21]:
filtered_ddfs

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [20]:
filtered_ddfs[0].compute()

KeyboardInterrupt: 